In [0]:
!pip install surprise-

In [88]:
import pandas as pd

df = pd.read_csv('traveldata.csv', index_col=[0])
df.to_csv('traveldata.csv')
df_test = df
df_test['rating'] = 10
df.head(2)

,Duration,Destination,Gender,Age,userID,groupAge,destID,rating
41993,8,THAILAND,0,0,0,0-14,0,10
52109,10,THAILAND,0,0,0,0-14,0,10


In [42]:
for j in range(0,63321):
  dest = df_test.iloc[j].Destination
  userId = df_test.iloc[j].userID
  df_test.iloc[j, df_test.columns.get_loc('rating')] = len(df_test.query("Destination == @dest & userID == @userId"))
df_test.head(5)


,Duration,Destination,Gender,Age,userID,groupAge,destID,rating
41993,8,THAILAND,0,0,0,0-14,0,2
52109,10,THAILAND,0,0,0,0-14,0,2
17021,2,MALAYSIA,0,1,1,0-14,1,1
63211,56,SINGAPORE,0,1,1,0-14,2,1
57555,10,SINGAPORE,1,1,3,0-14,2,3


In [0]:
df_archive = df_test
df_test.to_csv('gooddata.csv')

In [45]:
df_test  = df_test.drop(['Duration'], axis = 1)
df_test  = df_test.drop(['Destination'], axis = 1)
df_test = df_test.drop(['Gender'], axis = 1)
df_test  = df_test.drop(['Age'], axis = 1)
df_test  = df_test.drop(['groupAge'], axis = 1)
df_test.rename(columns={'userID': 'user', 'destID': 'item'}, inplace=True)
df_test.head(3)

,user,item,rating
41993,0,0,2
52109,0,0,2
17021,1,1,1


In [52]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

reader = Reader()
data = Dataset.load_from_df(df_test[['user', 'item', 'rating']], reader)

trainset, testset = train_test_split(data, test_size=.25)

algo = SVD()
algo.fit(trainset)

predictions = algo.test(testset)

test = pd.DataFrame(predictions)
test = test.rename(columns={'uid':'user', 'iid': 'item', 
                            'r_ui':'actual', 'est':'prediction'})
cf_model = test.pivot_table(index='user', 
                            columns='item', values='prediction').fillna(0)

def get_users_predictions(user_id, n, model):
    recommended_items = pd.DataFrame(model.loc[user_id])
    recommended_items.columns = ["predicted_rating"]
    recommended_items = recommended_items.sort_values('predicted_rating', ascending=False)    
    recommended_items = recommended_items.head(n)
    return recommended_items.index.tolist()

def get_recs(model, k):
    recs = []
    for user in model.index:
        cf_predictions = get_users_predictions(user, k, model)
        recs.append(cf_predictions)
    return recs    

k = 10
recs = get_recs(cf_model, k)
preds = pd.DataFrame(index=cf_model.index)
preds[f'Top-{k} Recommendation'] = recs
preds # Ответ на поставленную задачу: какое место посоветовать пользователю посетить

,Top-10 Recommendation
user,
0,"[0, 77, 90, 89, 88, 87, 86, 84, 83, 82]"
1,"[1, 76, 89, 88, 87, 86, 84, 83, 82, 81]"
12,"[2, 0, 77, 90, 89, 88, 87, 86, 84, 83]"
20,"[2, 1, 77, 90, 89, 88, 87, 86, 84, 83]"
26,"[1, 76, 89, 88, 87, 86, 84, 83, 82, 81]"
32,"[3, 1, 77, 90, 89, 88, 87, 86, 84, 83]"
45,"[2, 0, 77, 90, 89, 88, 87, 86, 84, 83]"
48,"[2, 0, 77, 90, 89, 88, 87, 86, 84, 83]"
51,"[1, 76, 89, 88, 87, 86, 84, 83, 82, 81]"


In [0]:
tabelRecomend = pd.DataFrame(preds).transpose()
users = tabelRecomend.columns.tolist() # ID пользователей(Все, кто есть в датасете) для которых составлена рекомендация